In [3]:
import zipfile
import os
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import defaultdict
import ast
from tqdm.notebook import tqdm as tqdm
import pickle

# Bigram Inverted Index

In [1]:
# helper function that generates all bigrams for a given set of tokens and accordingly updates the posting list and the doc frequency corresponding to the bigram.
def helper(inv_idx_freq, inv_idx_postings, list_of_tokens, docID):
    n = len(list_of_tokens)

    for i in range(n-1):
        token = (list_of_tokens[i], list_of_tokens[i+1])
        posting_list =  inv_idx_postings[token]
        if docID in posting_list:
            continue
        else:
            inv_idx_postings[token].append(docID)
            inv_idx_freq[token] = inv_idx_freq[token] + 1
    
    

In [ ]:
def t1_and_t2(posting1, posting2):
  
  # if any of the postings is empty then their intersection will also be an empty list
  if len(posting1) == 0 or len(posting2) == 0:
    return []

  ptr1 = 0 #pointer that iterates over posting 1
  ptr2 = 0 #pointer that iterates over posting 2

  comparisons = 0 #intialize number of comparison as 0
  ans = []


  while ptr1 < len(posting1) and ptr2 < len(posting2):
    comparisons += 1 
    if posting1[ptr1] ==  posting2[ptr2]: # this means that both the docs contains the word.
      ans.append(posting1[ptr1])
      ptr1 = ptr1 + 1
      ptr2 = ptr2 + 1

    elif posting1[ptr1] <  posting2[ptr2]:
      ptr1 = ptr1 + 1
    
    else:
      ptr2 = ptr2 + 1

  return ans, comparisons


In [ ]:
def not_t1(posting1):
    global docIDs
    ans = [] # to store all the docIDs except that are their in postings1

    for i in docIDs:
        if i in posting1:
            continue
        else:
            ans.append(i)
    return ans
    


In [ ]:
def t1_and_not_t2(posting1, posting2):
    ptr1 = 0 #pointer that iterates over posting 1
    ptr2 = 0 #pointer that iterates over posting 2

    comparisons = 0 #intialize number of comparison as 0
    ans = []

    while ptr1 < len(posting1) and ptr2 < len(posting2):
        comparisons += 1

        if posting1[ptr1] == posting2[ptr2]: 
            ptr1+=1
            ptr2+=1
            continue
        elif posting1[ptr1] < posting2[ptr2]:
            ans.append( posting1[ptr1] )
            ptr1+=1

        else:
            ptr2+=1

    while ptr1 < len(posting1):
        ans.append(posting1[ptr1])
        ptr1+=1

    return ans, comparisons

In [ ]:
def t1_or_t2(posting1, posting2):

    #if any of the posting list is empty we will return the other list.
    if len(posting1) == 0 :
        return posting2, 0
    if len(posting2) == 0:
        return posting1, 0
        
    ptr1 = 0 #pointer that iterates over posting 1
    ptr2 = 0 #pointer that iterates over posting 2

    comparisons = 0 #intialize number of comparison as 0
    ans = []

    while ptr1 < len(posting1) and ptr2 < len(posting2):
        comparisons+=1
        if posting1[ptr1] < posting2[ptr2]:
            ans.append(posting1[ptr1]) 
            ptr1+=1
        elif posting2[ptr2] < posting1[ptr1]  :
            ans.append(posting2[ptr2])
            ptr2+=1
        else:
            ans.append(posting2[ptr2])
            ptr1+=1
            ptr2+=1

    while(ptr1 < len(posting1)):
        ans.append( posting1[ptr1] )
        ptr1+=1
    
    while(ptr2 < len(posting2)):
        ans.append( posting2[ptr2] )
        ptr2+=1
    
    return ans, comparisons


        

In [ ]:
def t1_or_not_t2(posting1, posting2):
    # here we need to take the not of posting2 and then do an or with posting 1

    if len(posting1) == 0: #if the first posting list is empty we will return the not of the other list.
        return not_t1(posting2)
    
    not_posting2 = not_t1(posting2)

    ans, comparisons = t1_or_t2(posting1, not_posting2)
    return ans, comparisons

    
    

In [29]:
inv_idx_freq  = defaultdict(int) # initializing the doc Frequency dictionary as a default dict which return 0 whenver a key does not exists.
inv_idx_postings  = defaultdict(list) # intializing the postings dictionary to return an empty list whenever it is invoked for a key that does not exits.
sample = 2
docIDs = []
folder_path = 'Q1_2_output'
for filename in tqdm(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            contents = file.read()
            docID = int(filename[-4:])
            docIDs.append(docID)
            # print(docID)
            # set_of_tokens = set(contents.strip('][').split(','))
            list_of_tokens = ast.literal_eval(contents)
            helper(inv_idx_freq, inv_idx_postings, list_of_tokens, docID)
            # print(f'{filename}: {contents}')
            # print(set_of_tokens)
            # sample -= 1
            # if (sample == 0):
            #     break


#sort the index in alphabetical order wrt terms
inv_idx_postings = dict(sorted(inv_idx_postings.items()))
inv_idx_freq = dict(sorted(inv_idx_freq.items()))

for key in inv_idx_postings:
    inv_idx_postings[key].sort() #sort the indivdual posting lists correponging to each term in the index


  0%|          | 0/1400 [00:00<?, ?it/s]

In [23]:
# print(inv_idx_postings)

In [30]:
print(inv_idx_postings[('accurate', 'calculation')])

[455]


In [25]:
pickle.dump(inv_idx_postings, open('bi_inv_idx_postings.pkl', 'wb'))
pickle.dump(inv_idx_freq, open('bi_inv_idx_freq.pkl', 'wb'))

In [26]:
inv_idx_postings = pickle.load(open("bi_inv_idx_postings.pkl", 'rb'))
inv_idx_freq = pickle.load(open("bi_inv_idx_freq.pkl", 'rb'))

In [28]:
print(inv_idx_postings[('accurate', 'calculation')])

[455]


# Positional index